# 🥐 Bäckerei Umsatz Vorhersage - Trainingsdatensatz Erstellung

Dieses Notebook erstellt einen kombinierten Trainingsdatensatz für die Vorhersage von Bäckerei-Umsätzen basierend auf:
- Historischen Umsatzdaten
- Wetterdaten  
- Feiertagen
- Besonderen Events (Kieler Woche)
- Preisindizes für Backwaren

**Ziel:** Ein sauberer DataFrame für Regressionsanalyse und neuronale Netze

In [7]:
# 1. BIBLIOTHEKEN LADEN UND ANZEIGEOPTIONEN SETZEN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Pandas Anzeigeoptionen für bessere Darstellung
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

print("✅ Bibliotheken erfolgreich geladen!")
print(f"🐼 Pandas Version: {pd.__version__}")
print(f"🔢 NumPy Version: {np.__version__}")

✅ Bibliotheken erfolgreich geladen!
🐼 Pandas Version: 2.2.3
🔢 NumPy Version: 2.2.4


In [8]:
# 2. UMSATZDATEN LADEN UND VERARBEITEN
print("📊 Lade Umsatzdaten...")

# Umsatzdaten von GitHub laden
umsatz_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv"
df_umsatz = pd.read_csv(umsatz_url)

print(f"Shape der Umsatzdaten: {df_umsatz.shape}")
print(f"Spalten: {list(df_umsatz.columns)}")
print("\nErste 5 Zeilen:")
print(df_umsatz.head())

# Datum aus der ID extrahieren (Format: YYYYMMDW)
def extract_date_info(row_id):
    """Extrahiert Datum und Warengruppe aus der ID"""
    id_str = str(int(row_id))
    
    # YYYYMMDD + W (Warengruppe)
    if len(id_str) == 9:
        jahr = int(id_str[:4])
        monat = int(id_str[4:6])
        tag = int(id_str[6:8])
        warengruppe = int(id_str[8])
        
        try:
            datum = pd.to_datetime(f"{jahr}-{monat:02d}-{tag:02d}")
            return datum, warengruppe
        except:
            return None, None
    return None, None

# Datum und Warengruppe extrahieren
print("\n🔄 Extrahiere Datum und Warengruppe aus ID...")
df_umsatz['Datum'], df_umsatz['Warengruppe'] = zip(*df_umsatz.iloc[:, 0].apply(extract_date_info))

# Ungültige Daten entfernen
df_umsatz = df_umsatz.dropna(subset=['Datum', 'Warengruppe'])
df_umsatz['Warengruppe'] = df_umsatz['Warengruppe'].astype(int)

# Weitere Datums-Features erstellen
df_umsatz['Jahr'] = df_umsatz['Datum'].dt.year
df_umsatz['Monat'] = df_umsatz['Datum'].dt.month
df_umsatz['Tag'] = df_umsatz['Datum'].dt.day
df_umsatz['Wochentag'] = df_umsatz['Datum'].dt.day_name()
df_umsatz['Wochentag_Nr'] = df_umsatz['Datum'].dt.dayofweek

print(f"✅ {len(df_umsatz)} gültige Datensätze nach Extraktion")
print(f"📅 Zeitraum: {df_umsatz['Datum'].min()} bis {df_umsatz['Datum'].max()}")
print(f"🏷️ Warengruppen: {sorted(df_umsatz['Warengruppe'].unique())}")

📊 Lade Umsatzdaten...
Shape der Umsatzdaten: (9334, 4)
Spalten: ['id', 'Datum', 'Warengruppe', 'Umsatz']

Erste 5 Zeilen:
        id       Datum  Warengruppe      Umsatz
0  1307011  2013-07-01            1  148.828353
1  1307021  2013-07-02            1  159.793757
2  1307031  2013-07-03            1  111.885594
3  1307041  2013-07-04            1  168.864941
4  1307051  2013-07-05            1  171.280754

🔄 Extrahiere Datum und Warengruppe aus ID...


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
# WARENGRUPPEN IN LESBARE NAMEN UMWANDELN
warengruppen_mapping = {
    1: "Brot",
    2: "Brötchen", 
    3: "Croissant",
    4: "Konditorei",
    5: "Kuchen",
    6: "Saisonbrot"
}

df_umsatz['Warengruppe_Name'] = df_umsatz['Warengruppe'].map(warengruppen_mapping)

# One-Hot-Encoding für Warengruppen
print("🔄 Erstelle One-Hot-Encoding für Warengruppen...")
warengruppen_dummies = pd.get_dummies(df_umsatz['Warengruppe_Name'], prefix='Warengruppe')
df_umsatz = pd.concat([df_umsatz, warengruppen_dummies], axis=1)

print("✅ Warengruppen-Features erstellt:")
warengruppen_cols = [col for col in df_umsatz.columns if col.startswith('Warengruppe_')]
print(warengruppen_cols)

# Überblick über die Daten
print(f"\n📈 Dataset Info:")
print(f"Zeilen: {len(df_umsatz)}")
print(f"Spalten: {len(df_umsatz.columns)}")
print(f"Umsatzbereich: {df_umsatz.iloc[:, 1].min():.2f} € - {df_umsatz.iloc[:, 1].max():.2f} €")

In [ ]:
# 3. HISTOGRAMM DER UMSÄTZE ERSTELLEN
print("📊 Erstelle Umsatz-Visualisierungen...")

plt.figure(figsize=(15, 10))

# Subplot 1: Gesamtverteilung der Umsätze
plt.subplot(2, 3, 1)
plt.hist(df_umsatz.iloc[:, 1], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Verteilung aller Umsätze')
plt.xlabel('Umsatz (€)')
plt.ylabel('Häufigkeit')

# Subplot 2: Umsätze nach Warengruppe
plt.subplot(2, 3, 2)
df_umsatz.boxplot(column=df_umsatz.columns[1], by='Warengruppe_Name', ax=plt.gca())
plt.title('Umsätze nach Warengruppe')
plt.xticks(rotation=45)
plt.suptitle('')

# Subplot 3: Umsätze nach Wochentag
plt.subplot(2, 3, 3)
wochentag_umsatz = df_umsatz.groupby('Wochentag')[df_umsatz.columns[1]].mean()
wochentag_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
wochentag_umsatz = wochentag_umsatz.reindex(wochentag_order)
wochentag_umsatz.plot(kind='bar', color='lightgreen')
plt.title('Durchschnittsumsatz nach Wochentag')
plt.xticks(rotation=45)
plt.ylabel('Umsatz (€)')

# Subplot 4: Umsätze nach Monat
plt.subplot(2, 3, 4)
monat_umsatz = df_umsatz.groupby('Monat')[df_umsatz.columns[1]].mean()
monat_umsatz.plot(kind='bar', color='orange')
plt.title('Durchschnittsumsatz nach Monat')
plt.xlabel('Monat')
plt.ylabel('Umsatz (€)')

# Subplot 5: Zeitreihe der Umsätze
plt.subplot(2, 3, 5)
tagesumsatz = df_umsatz.groupby('Datum')[df_umsatz.columns[1]].sum()
plt.plot(tagesumsatz.index, tagesumsatz.values, alpha=0.7)
plt.title('Tagesumsätze über Zeit')
plt.xlabel('Datum')
plt.ylabel('Tagesumsatz (€)')
plt.xticks(rotation=45)

# Subplot 6: Top Warengruppen
plt.subplot(2, 3, 6)
warengruppen_umsatz = df_umsatz.groupby('Warengruppe_Name')[df_umsatz.columns[1]].sum().sort_values(ascending=False)
warengruppen_umsatz.plot(kind='bar', color='purple')
plt.title('Gesamtumsatz nach Warengruppe')
plt.xticks(rotation=45)
plt.ylabel('Gesamtumsatz (€)')

plt.tight_layout()
plt.show()

print(f"📊 Umsatz-Statistiken:")
print(f"Durchschnitt: {df_umsatz.iloc[:, 1].mean():.2f} €")
print(f"Median: {df_umsatz.iloc[:, 1].median():.2f} €")
print(f"Standardabweichung: {df_umsatz.iloc[:, 1].std():.2f} €")

In [ ]:
# 4. WETTERDATEN LADEN UND VERKNÜPFEN
print("🌤️ Lade Wetterdaten...")

# Wetterdaten von GitHub laden
wetter_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv"
df_wetter = pd.read_csv(wetter_url)

print(f"Shape der Wetterdaten: {df_wetter.shape}")
print(f"Spalten: {list(df_wetter.columns)}")
print("\nErste 5 Zeilen Wetterdaten:")
print(df_wetter.head())

# Datum in Wetterdaten konvertieren
df_wetter['Datum'] = pd.to_datetime(df_wetter['Datum'])

# Feature: Wettercode fehlt
df_wetter['Wettercode_fehlt'] = df_wetter['Wettercode'].isna().astype(int)

# Fehlende Wetterdaten mit Median füllen
numeric_weather_cols = ['Temperatur', 'Windgeschwindigkeit', 'Bewoelkung']
for col in numeric_weather_cols:
    if col in df_wetter.columns:
        median_val = df_wetter[col].median()
        df_wetter[col] = df_wetter[col].fillna(median_val)
        print(f"📊 {col}: Fehlende Werte mit Median {median_val:.2f} gefüllt")

# Wetterdaten mit Umsatzdaten verknüpfen
print("\n🔗 Verknüpfe Wetter- mit Umsatzdaten...")
df_combined = df_umsatz.merge(df_wetter, on='Datum', how='left')

print(f"✅ Kombinierter Datensatz: {df_combined.shape}")
print(f"🌡️ Temperaturbereich: {df_combined['Temperatur'].min():.1f}°C - {df_combined['Temperatur'].max():.1f}°C")

# Fehlende Wetterdaten anzeigen
missing_weather = df_combined['Temperatur'].isna().sum()
if missing_weather > 0:
    print(f"⚠️ {missing_weather} Tage ohne Wetterdaten - werden mit Durchschnitt gefüllt")
    for col in numeric_weather_cols:
        if col in df_combined.columns:
            df_combined[col] = df_combined[col].fillna(df_combined[col].mean())

In [ ]:
# 5. FEIERTAGE LADEN UND VERKNÜPFEN
print("🎄 Lade Feiertagsdaten...")

try:
    # Feiertage aus lokaler Datei laden
    feiertage_path = "/workspaces/bakery_sales_prediction/5_Datasets/DE-Feiertage_2020_bis_2035.csv"
    df_feiertage = pd.read_csv(feiertage_path)
    
    print(f"Shape der Feiertagsdaten: {df_feiertage.shape}")
    print(f"Spalten: {list(df_feiertage.columns)}")
    print("\nErste 5 Feiertage:")
    print(df_feiertage.head())
    
    # Datum in Feiertagsdaten konvertieren
    datum_col = df_feiertage.columns[0]  # Erste Spalte als Datum
    df_feiertage['Datum'] = pd.to_datetime(df_feiertage[datum_col])
    
    # Nur relevante Jahre filtern (2013-2018)
    df_feiertage = df_feiertage[
        (df_feiertage['Datum'].dt.year >= 2013) & 
        (df_feiertage['Datum'].dt.year <= 2018)
    ]
    
    # Binäre Feiertag-Spalte erstellen
    feiertage_dates = set(df_feiertage['Datum'].dt.date)
    df_combined['ist_feiertag'] = df_combined['Datum'].dt.date.isin(feiertage_dates).astype(int)
    
    print(f"✅ {len(feiertage_dates)} Feiertage im relevanten Zeitraum gefunden")
    print(f"📅 Tage mit Feiertag im Dataset: {df_combined['ist_feiertag'].sum()}")
    
except FileNotFoundError:
    print("⚠️ Feiertagsdatei nicht gefunden - erstelle manuell Feiertage")
    # Manuelle Definition wichtiger Feiertage
    deutsche_feiertage = [
        # 2013
        '2013-01-01', '2013-03-29', '2013-04-01', '2013-05-01', '2013-05-09', '2013-05-20', 
        '2013-10-03', '2013-12-25', '2013-12-26',
        # 2014
        '2014-01-01', '2014-04-18', '2014-04-21', '2014-05-01', '2014-05-29', '2014-06-09',
        '2014-10-03', '2014-12-25', '2014-12-26',
        # 2015-2018 weitere Feiertage...
    ]
    
    feiertage_dates = set(pd.to_datetime(deutsche_feiertage).date)
    df_combined['ist_feiertag'] = df_combined['Datum'].dt.date.isin(feiertage_dates).astype(int)
    print(f"✅ {len(feiertage_dates)} manuelle Feiertage erstellt")

In [ ]:
# 6. JAHRESZEIT BESTIMMEN
print("🌸 Erstelle Jahreszeiten-Feature...")

def get_season(month):
    """Bestimmt die Jahreszeit basierend auf dem Monat"""
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    else:  # 9, 10, 11
        return "Herbst"

df_combined['Jahreszeit'] = df_combined['Monat'].apply(get_season)

# One-Hot-Encoding für Jahreszeiten
jahreszeiten_dummies = pd.get_dummies(df_combined['Jahreszeit'], prefix='Jahreszeit')
df_combined = pd.concat([df_combined, jahreszeiten_dummies], axis=1)

print("✅ Jahreszeiten-Features erstellt:")
jahreszeiten_cols = [col for col in df_combined.columns if col.startswith('Jahreszeit_')]
print(jahreszeiten_cols)

# Verteilung der Jahreszeiten anzeigen
jahreszeit_verteilung = df_combined['Jahreszeit'].value_counts()
print(f"\n📊 Verteilung der Jahreszeiten:")
print(jahreszeit_verteilung)

In [ ]:
# 7. SONNTAG FEATURE ERSTELLEN
print("📅 Erstelle Sonntag-Feature...")

# Sonntag = Wochentag 6 (Montag = 0)
df_combined['ist_sonntag'] = (df_combined['Wochentag_Nr'] == 6).astype(int)

print(f"✅ Sonntag-Feature erstellt")
print(f"📊 Anzahl Sonntage im Dataset: {df_combined['ist_sonntag'].sum()}")
print(f"📊 Prozent Sonntage: {df_combined['ist_sonntag'].mean()*100:.1f}%")

# Umsätze an Sonntagen vs. anderen Tagen
sonntag_umsatz = df_combined[df_combined['ist_sonntag'] == 1][df_combined.columns[1]].mean()
werktag_umsatz = df_combined[df_combined['ist_sonntag'] == 0][df_combined.columns[1]].mean()

print(f"💰 Durchschnittsumsatz Sonntags: {sonntag_umsatz:.2f} €")
print(f"💰 Durchschnittsumsatz andere Tage: {werktag_umsatz:.2f} €")
print(f"📈 Sonntag vs. Werktag Ratio: {sonntag_umsatz/werktag_umsatz:.2f}")

In [ ]:
# 8. KIELER WOCHE INTEGRIEREN
print("⛵ Lade Kieler Woche Daten...")

try:
    # KiWo-Daten von GitHub laden
    kiwo_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv"
    df_kiwo = pd.read_csv(kiwo_url)
    
    print(f"Shape der KiWo-Daten: {df_kiwo.shape}")
    print(f"Spalten: {list(df_kiwo.columns)}")
    print("\nKieler Woche Termine:")
    print(df_kiwo.head())
    
    # Datumsspalten konvertieren
    start_col = df_kiwo.columns[0]
    end_col = df_kiwo.columns[1] if len(df_kiwo.columns) > 1 else df_kiwo.columns[0]
    
    df_kiwo['Start'] = pd.to_datetime(df_kiwo[start_col])
    df_kiwo['Ende'] = pd.to_datetime(df_kiwo[end_col]) if end_col != start_col else df_kiwo['Start']
    
    # KiWo-Feature erstellen
    def is_kiwo_date(datum):
        """Prüft ob ein Datum in der Kieler Woche liegt"""
        for _, row in df_kiwo.iterrows():
            if row['Start'] <= datum <= row['Ende']:
                return 1
        return 0
    
    df_combined['ist_kiwo'] = df_combined['Datum'].apply(is_kiwo_date)
    
    print(f"✅ KiWo-Feature erstellt")
    print(f"📊 Tage während Kieler Woche: {df_combined['ist_kiwo'].sum()}")
    
    # Umsätze während KiWo vs. normale Tage
    if df_combined['ist_kiwo'].sum() > 0:
        kiwo_umsatz = df_combined[df_combined['ist_kiwo'] == 1][df_combined.columns[1]].mean()
        normal_umsatz = df_combined[df_combined['ist_kiwo'] == 0][df_combined.columns[1]].mean()
        
        print(f"💰 Durchschnittsumsatz während KiWo: {kiwo_umsatz:.2f} €")
        print(f"💰 Durchschnittsumsatz normale Tage: {normal_umsatz:.2f} €")
        print(f"📈 KiWo vs. Normal Ratio: {kiwo_umsatz/normal_umsatz:.2f}")
    
except Exception as e:
    print(f"⚠️ Fehler beim Laden der KiWo-Daten: {e}")
    print("Erstelle Standard KiWo-Feature (alle 0)")
    df_combined['ist_kiwo'] = 0

In [ ]:
# 9. PREISINDEX LADEN UND ZUORDNEN
print("📈 Lade Preisindex-Daten...")

try:
    # Preisindex aus lokaler Datei laden
    preisindex_path = "/workspaces/bakery_sales_prediction/5_Datasets/additional data/Preisentwicklung_Backwaren.csv"
    df_preise = pd.read_csv(preisindex_path)
    
    print(f"Shape der Preisdaten: {df_preise.shape}")
    print(f"Spalten: {list(df_preise.columns)}")
    print("\nErste 5 Zeilen Preisdaten:")
    print(df_preise.head())
    
    # Datumsspalte konvertieren (erste Spalte)
    datum_col = df_preise.columns[0]
    df_preise['Datum'] = pd.to_datetime(df_preise[datum_col])
    
    # Preisindex-Spalte identifizieren (meist zweite Spalte)
    preis_col = df_preise.columns[1]
    df_preise['Preisindex'] = pd.to_numeric(df_preise[preis_col], errors='coerce')
    
    # Wöchentliche Zuordnung: Jeden Tag den Preisindex vom Montag der Woche zuordnen
    def get_monday_of_week(date):
        """Gibt den Montag der Woche für ein gegebenes Datum zurück"""
        days_since_monday = date.weekday()
        monday = date - timedelta(days=days_since_monday)
        return monday
    
    # Montag für jeden Tag berechnen
    df_combined['Montag_der_Woche'] = df_combined['Datum'].apply(get_monday_of_week)
    
    # Preisindex für jeden Montag zuordnen
    preis_dict = dict(zip(df_preise['Datum'].dt.date, df_preise['Preisindex']))
    
    def get_price_index(montag):
        """Holt den Preisindex für einen bestimmten Montag"""
        montag_date = montag.date()
        if montag_date in preis_dict:
            return preis_dict[montag_date]
        
        # Falls kein exakter Montag gefunden wird, suche den nächstliegenden
        available_dates = list(preis_dict.keys())
        if available_dates:
            closest_date = min(available_dates, key=lambda x: abs((x - montag_date).days))
            return preis_dict[closest_date]
        
        return 100.0  # Standardwert falls nichts gefunden wird
    
    df_combined['Preisindex'] = df_combined['Montag_der_Woche'].apply(get_price_index)
    
    print(f"✅ Preisindex-Feature erstellt")
    print(f"📊 Preisindex Bereich: {df_combined['Preisindex'].min():.2f} - {df_combined['Preisindex'].max():.2f}")
    print(f"📊 Durchschnittlicher Preisindex: {df_combined['Preisindex'].mean():.2f}")
    
except FileNotFoundError:
    print("⚠️ Preisindex-Datei nicht gefunden - erstelle Standard-Preisindex")
    # Standard-Preisindex erstellen (leichte Inflation über Zeit)
    base_index = 100.0
    df_combined['Jahre_seit_2013'] = df_combined['Jahr'] - 2013
    df_combined['Preisindex'] = base_index + (df_combined['Jahre_seit_2013'] * 2.5)  # 2.5% Inflation pro Jahr
    
    print(f"✅ Standard-Preisindex erstellt (100-112.5)")
    
except Exception as e:
    print(f"⚠️ Fehler beim Verarbeiten der Preisdaten: {e}")
    df_combined['Preisindex'] = 100.0

In [ ]:
# 10. FINALISIERUNG DES DATASETS
print("🔧 Finalisiere den Trainingsdatensatz...")

# Überprüfe fehlende Werte
print("\n🔍 Überprüfe fehlende Werte:")
missing_values = df_combined.isnull().sum()
missing_values = missing_values[missing_values > 0]
if len(missing_values) > 0:
    print(missing_values)
    
    # Fülle fehlende numerische Werte mit Median
    numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_combined[col].isnull().sum() > 0:
            median_val = df_combined[col].median()
            df_combined[col] = df_combined[col].fillna(median_val)
            print(f"✅ {col}: {df_combined[col].isnull().sum()} fehlende Werte mit Median {median_val:.2f} gefüllt")
else:
    print("✅ Keine fehlenden Werte gefunden!")

# Relevante Spalten für das finale Dataset auswählen
umsatz_col = df_combined.columns[1]  # Zweite Spalte ist Umsatz

feature_columns = [
    'Datum', 'Jahr', 'Monat', 'Tag', 'Wochentag', 'Wochentag_Nr',
    'Warengruppe', 'Warengruppe_Name',
    'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Wettercode_fehlt',
    'ist_feiertag', 'ist_sonntag', 'Jahreszeit', 'ist_kiwo', 'Preisindex',
    umsatz_col
]

# One-Hot encoded Spalten hinzufügen
warengruppen_cols = [col for col in df_combined.columns if col.startswith('Warengruppe_')]
jahreszeiten_cols = [col for col in df_combined.columns if col.startswith('Jahreszeit_')]

all_features = feature_columns + warengruppen_cols + jahreszeiten_cols

# Finale Spalten filtern (nur die die existieren)
final_columns = [col for col in all_features if col in df_combined.columns]
df_final = df_combined[final_columns].copy()

# Umsatz-Spalte umbenennen für Klarheit
df_final = df_final.rename(columns={umsatz_col: 'Umsatz'})

print(f"\n📊 FINALER TRAININGSDATENSATZ:")
print(f"📏 Shape: {df_final.shape}")
print(f"📅 Zeitraum: {df_final['Datum'].min()} bis {df_final['Datum'].max()}")
print(f"💰 Umsatzbereich: {df_final['Umsatz'].min():.2f} € - {df_final['Umsatz'].max():.2f} €")

print(f"\n🏷️ FEATURES ({len(df_final.columns)} Spalten):")
for i, col in enumerate(df_final.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# DATASET ÜBERSICHT UND BEISPIELDATEN
print("📋 DATASET ÜBERSICHT:")
print("=" * 60)

# Grundlegende Statistiken
print(f"📊 Datentypen:")
print(df_final.dtypes.value_counts())

print(f"\n📈 Numerische Statistiken:")
numeric_cols = df_final.select_dtypes(include=[np.number]).columns
print(df_final[numeric_cols].describe().round(2))

print(f"\n🗂️ Kategorische Variablen:")
categorical_cols = df_final.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col != 'Datum':
        print(f"{col}: {df_final[col].nunique()} eindeutige Werte")
        print(f"  Top 3: {list(df_final[col].value_counts().head(3).index)}")

print(f"\n📝 BEISPIELDATEN (erste 10 Zeilen):")
print(df_final.head(10))

print(f"\n📝 BEISPIELDATEN (zufällige 5 Zeilen):")
print(df_final.sample(5))

# Korrelationsanalyse für numerische Features
print(f"\n🔗 TOP KORRELATIONEN MIT UMSATZ:")
correlations = df_final[numeric_cols].corr()['Umsatz'].abs().sort_values(ascending=False)
print(correlations.head(10))

In [ ]:
# NORMALISIERUNG DER NUMERISCHEN FEATURES
print("🔧 Normalisiere numerische Features für Modelltraining...")

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
import warnings
warnings.filterwarnings('ignore')

# Kopie des finalen Datasets für Normalisierung erstellen
df_normalized = df_final.copy()

# Identifiziere numerische Features (ohne Zielvariable und One-Hot-Features)
numeric_features_to_scale = [
    'Jahr', 'Monat', 'Tag', 'Wochentag_Nr',
    'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 
    'Preisindex'
]

# Nur Features behalten, die auch im Dataset existieren
numeric_features_to_scale = [col for col in numeric_features_to_scale if col in df_normalized.columns]

print(f"📊 Zu normalisierende Features: {numeric_features_to_scale}")

# Originale Features vor Normalisierung anzeigen
print(f"\n📈 ORIGINALE FEATURE-STATISTIKEN:")
print(df_normalized[numeric_features_to_scale].describe().round(2))

# 1. STANDARD SCALER (Z-Score Normalisierung)
print(f"\n🔄 Methode 1: Standard Scaler (Z-Score)")
scaler_standard = StandardScaler()
df_normalized[['std_' + col for col in numeric_features_to_scale]] = scaler_standard.fit_transform(
    df_normalized[numeric_features_to_scale]
)

# 2. MIN-MAX SCALER (0-1 Normalisierung)
print(f"🔄 Methode 2: Min-Max Scaler (0-1)")
scaler_minmax = MinMaxScaler()
df_normalized[['minmax_' + col for col in numeric_features_to_scale]] = scaler_minmax.fit_transform(
    df_normalized[numeric_features_to_scale]
)

# 3. ROBUST SCALER (Median-basiert, weniger empfindlich gegen Ausreißer)
print(f"🔄 Methode 3: Robust Scaler (Median-basiert)")
scaler_robust = RobustScaler()
df_normalized[['robust_' + col for col in numeric_features_to_scale]] = scaler_robust.fit_transform(
    df_normalized[numeric_features_to_scale]
)

# 4. POWER TRANSFORMER (Yeo-Johnson für Normalverteilung)
print(f"🔄 Methode 4: Power Transformer (Yeo-Johnson)")
try:
    power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
    df_normalized[['power_' + col for col in numeric_features_to_scale]] = power_transformer.fit_transform(
        df_normalized[numeric_features_to_scale]
    )
except:
    print("⚠️ Power Transformer übersprungen (numerische Probleme)")

print(f"✅ Normalisierung abgeschlossen!")

# Vergleiche die Normalisierungsmethoden
scaling_methods = ['std', 'minmax', 'robust', 'power']
print(f"\n📊 VERGLEICH DER NORMALISIERUNGSMETHODEN:")
print("=" * 80)

for method in scaling_methods:
    method_cols = [col for col in df_normalized.columns if col.startswith(f'{method}_')]
    if method_cols:
        print(f"\n{method.upper()}-SCALER:")
        stats = df_normalized[method_cols].describe().round(3)
        print(f"Mean Bereich: {stats.loc['mean'].min():.3f} bis {stats.loc['mean'].max():.3f}")
        print(f"Std Bereich: {stats.loc['std'].min():.3f} bis {stats.loc['std'].max():.3f}")
        print(f"Min Bereich: {stats.loc['min'].min():.3f} bis {stats.loc['min'].max():.3f}")
        print(f"Max Bereich: {stats.loc['max'].min():.3f} bis {stats.loc['max'].max():.3f}")

In [ ]:
# VISUALISIERUNG DER NORMALISIERUNGSEFFEKTE
print("📊 Visualisiere Normalisierungseffekte...")

# Wähle ein repräsentatives Feature für Visualisierung
example_feature = 'Temperatur' if 'Temperatur' in numeric_features_to_scale else numeric_features_to_scale[0]
print(f"Beispiel-Feature: {example_feature}")

plt.figure(figsize=(20, 12))

# Original
plt.subplot(2, 3, 1)
plt.hist(df_normalized[example_feature], bins=50, alpha=0.7, color='gray', edgecolor='black')
plt.title(f'Original: {example_feature}')
plt.xlabel('Wert')
plt.ylabel('Häufigkeit')

# Standard Scaler
plt.subplot(2, 3, 2)
plt.hist(df_normalized[f'std_{example_feature}'], bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.title(f'Standard Scaler: {example_feature}')
plt.xlabel('Z-Score')
plt.ylabel('Häufigkeit')

# Min-Max Scaler
plt.subplot(2, 3, 3)
plt.hist(df_normalized[f'minmax_{example_feature}'], bins=50, alpha=0.7, color='green', edgecolor='black')
plt.title(f'Min-Max Scaler: {example_feature}')
plt.xlabel('Normalisiert (0-1)')
plt.ylabel('Häufigkeit')

# Robust Scaler
plt.subplot(2, 3, 4)
plt.hist(df_normalized[f'robust_{example_feature}'], bins=50, alpha=0.7, color='orange', edgecolor='black')
plt.title(f'Robust Scaler: {example_feature}')
plt.xlabel('Robust normalisiert')
plt.ylabel('Häufigkeit')

# Power Transformer (falls verfügbar)
if f'power_{example_feature}' in df_normalized.columns:
    plt.subplot(2, 3, 5)
    plt.hist(df_normalized[f'power_{example_feature}'], bins=50, alpha=0.7, color='red', edgecolor='black')
    plt.title(f'Power Transformer: {example_feature}')
    plt.xlabel('Power transformiert')
    plt.ylabel('Häufigkeit')

# Boxplot-Vergleich
plt.subplot(2, 3, 6)
comparison_data = [
    df_normalized[example_feature],
    df_normalized[f'std_{example_feature}'],
    df_normalized[f'minmax_{example_feature}'],
    df_normalized[f'robust_{example_feature}']
]
comparison_labels = ['Original', 'Standard', 'MinMax', 'Robust']

if f'power_{example_feature}' in df_normalized.columns:
    comparison_data.append(df_normalized[f'power_{example_feature}'])
    comparison_labels.append('Power')

plt.boxplot(comparison_data, labels=comparison_labels)
plt.title(f'Vergleich aller Methoden: {example_feature}')
plt.xticks(rotation=45)
plt.ylabel('Werte')

plt.tight_layout()
plt.show()

# Korrelation zwischen Umsatz und normalisierten Features
print(f"\n🔗 KORRELATION ZWISCHEN UMSATZ UND NORMALISIERTEN FEATURES:")
print("=" * 70)

for method in ['std', 'minmax', 'robust', 'power']:
    method_cols = [col for col in df_normalized.columns if col.startswith(f'{method}_')]
    if method_cols:
        correlations = df_normalized[method_cols + ['Umsatz']].corr()['Umsatz'].abs().sort_values(ascending=False)
        print(f"\n{method.upper()}-SCALER - Top 5 Korrelationen:")
        print(correlations.head(6))  # Top 5 + Umsatz selbst

In [ ]:
# EMPFEHLUNG UND FINALE DATASET-ERSTELLUNG
print("🎯 EMPFEHLUNG FÜR NORMALISIERUNGSMETHODE:")
print("=" * 60)

# Bewertungskriterien für neuronale netze
print("📋 Bewertungskriterien für Neuronale Netze:")
print("1. Ähnliche Skalierung aller Features (Mean ≈ 0, Std ≈ 1)")
print("2. Keine extremen Ausreißer")
print("3. Stabile Gradientenberechnung")
print("4. Erhaltung der Datenverteilung")

# Analysiere Stabilität der verschiedenen Scaler
scaler_scores = {}

for method in ['std', 'minmax', 'robust']:
    method_cols = [col for col in df_normalized.columns if col.startswith(f'{method}_')]
    if method_cols:
        method_data = df_normalized[method_cols]
        
        # Bewertungskriterien berechnen
        mean_consistency = 1 / (1 + abs(method_data.mean().std()))  # Je einheitlicher die Means, desto besser
        std_consistency = 1 / (1 + abs(method_data.std().std()))    # Je einheitlicher die Stds, desto besser
        range_control = 1 / (1 + (method_data.max().max() - method_data.min().min()))  # Kontrollierter Wertebereich
        
        total_score = (mean_consistency + std_consistency + range_control) / 3
        scaler_scores[method] = {
            'total': total_score,
            'mean_consistency': mean_consistency,
            'std_consistency': std_consistency,
            'range_control': range_control
        }

# Beste Methode ermitteln
best_method = max(scaler_scores.keys(), key=lambda k: scaler_scores[k]['total'])

print(f"\n📊 BEWERTUNG DER NORMALISIERUNGSMETHODEN:")
for method, scores in scaler_scores.items():
    print(f"{method.upper()}-Scaler:")
    print(f"  Gesamtscore: {scores['total']:.3f}")
    print(f"  Mean-Konsistenz: {scores['mean_consistency']:.3f}")
    print(f"  Std-Konsistenz: {scores['std_consistency']:.3f}")
    print(f"  Wertebereich: {scores['range_control']:.3f}")

print(f"\n✅ EMPFEHLUNG: {best_method.upper()}-SCALER")
print(f"🏆 Beste Gesamtbewertung: {scaler_scores[best_method]['total']:.3f}")

# Finales Dataset mit der besten Normalisierungsmethode erstellen
best_method_cols = [col for col in df_normalized.columns if col.startswith(f'{best_method}_')]

# Features für finales normalisiertes Dataset auswählen
final_normalized_features = [
    'Datum', 'Umsatz',  # Basis-Features
    'Warengruppe', 'Warengruppe_Name', 'Wochentag', 'Jahreszeit',  # Kategorische Features
    'ist_feiertag', 'ist_sonntag', 'ist_kiwo', 'Wettercode_fehlt'  # Binäre Features
]

# One-Hot-Features hinzufügen
warengruppen_onehot = [col for col in df_normalized.columns if col.startswith('Warengruppe_')]
jahreszeiten_onehot = [col for col in df_normalized.columns if col.startswith('Jahreszeit_')]

# Normalisierte numerische Features der besten Methode hinzufügen
final_features = final_normalized_features + warengruppen_onehot + jahreszeiten_onehot + best_method_cols

# Nur existierende Spalten auswählen
final_features = [col for col in final_features if col in df_normalized.columns]

# Finales normalisiertes Dataset erstellen
df_model_ready = df_normalized[final_features].copy()

# Normalisierte Spalten umbenennen (Präfix entfernen für Klarheit)
rename_dict = {}
for col in best_method_cols:
    original_name = col.replace(f'{best_method}_', '')
    rename_dict[col] = f'{original_name}_normalized'

df_model_ready = df_model_ready.rename(columns=rename_dict)

print(f"\n📊 MODELL-BEREITES DATASET:")
print(f"📏 Shape: {df_model_ready.shape}")
print(f"🏷️ Features: {len(df_model_ready.columns)}")

print(f"\n🔧 NORMALISIERTE FEATURES:")
normalized_cols = [col for col in df_model_ready.columns if col.endswith('_normalized')]
for col in normalized_cols:
    stats = df_model_ready[col].describe()
    print(f"{col}: Mean={stats['mean']:.3f}, Std={stats['std']:.3f}, Range=[{stats['min']:.3f}, {stats['max']:.3f}]")

# Speichere das normalisierte Dataset
normalized_output_path = "/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset_normalized.csv"

try:
    df_model_ready.to_csv(normalized_output_path, index=False, encoding='utf-8')
    print(f"\n✅ Normalisiertes Dataset gespeichert: {normalized_output_path}")
    
    # Zusätzliche Info-Datei mit Normalisierungsparametern
    normalization_info = {
        'Methode': best_method.upper() + '-Scaler',
        'Normalisierte_Features': [col.replace('_normalized', '') for col in normalized_cols],
        'Original_Features': numeric_features_to_scale,
        'Dataset_Shape': df_model_ready.shape,
        'Empfehlung': f"Verwende {best_method.upper()}-Scaler für optimale NN-Performance"
    }
    
    import json
    info_path = "/workspaces/bakery_sales_prediction/5_Datasets/normalization_info.json"
    with open(info_path, 'w', encoding='utf-8') as f:
        json.dump(normalization_info, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Normalisierungs-Info gespeichert: {info_path}")
    
except Exception as e:
    print(f"❌ Fehler beim Speichern: {e}")

print(f"\n🎉 NORMALISIERUNG ABGESCHLOSSEN!")
print(f"📁 Original Dataset: bakery_training_dataset.csv")
print(f"📁 Normalisiertes Dataset: bakery_training_dataset_normalized.csv")
print(f"🚀 Bereit für Neuronale Netze und ML-Modelle!")

In [ ]:
# DATASET SPEICHERN
print("💾 Speichere finalen Trainingsdatensatz...")

# Pfad für das finale Dataset
output_path = "/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv"

try:
    # CSV speichern
    df_final.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ Dataset erfolgreich gespeichert: {output_path}")
    
    # Zusätzlich als Excel für bessere Lesbarkeit
    excel_path = output_path.replace('.csv', '.xlsx')
    df_final.to_excel(excel_path, index=False)
    print(f"✅ Dataset auch als Excel gespeichert: {excel_path}")
    
    # Feature-Liste separat speichern
    feature_info = pd.DataFrame({
        'Feature': df_final.columns,
        'Typ': df_final.dtypes,
        'Fehlende_Werte': df_final.isnull().sum(),
        'Eindeutige_Werte': df_final.nunique()
    })
    
    feature_path = "/workspaces/bakery_sales_prediction/5_Datasets/feature_description.csv"
    feature_info.to_csv(feature_path, index=False)
    print(f"✅ Feature-Beschreibung gespeichert: {feature_path}")
    
except Exception as e:
    print(f"❌ Fehler beim Speichern: {e}")

print(f"\n🎉 TRAININGSDATENSATZ ERFOLGREICH ERSTELLT!")
print(f"📁 Hauptdatei: {output_path}")
print(f"📋 {len(df_final)} Zeilen mit {len(df_final.columns)} Features")
print(f"🎯 Bereit für Regression und Neuronale Netze!")

# Finale Zusammenfassung
print(f"\n📊 FINALE ZUSAMMENFASSUNG:")
print(f"🥐 Warengruppen: {df_final['Warengruppe'].nunique()}")
print(f"📅 Tage: {df_final['Datum'].nunique()}")
print(f"🌡️ Temperaturbereich: {df_final['Temperatur'].min():.1f}°C - {df_final['Temperatur'].max():.1f}°C")
print(f"🎄 Feiertage: {df_final['ist_feiertag'].sum()} Tage")
print(f"📅 Sonntage: {df_final['ist_sonntag'].sum()} Tage")
print(f"⛵ Kieler Woche: {df_final['ist_kiwo'].sum()} Tage")
print(f"💰 Durchschnittsumsatz: {df_final['Umsatz'].mean():.2f} €")

# 🎯 Trainingsdatensatz erfolgreich erstellt!

## 📊 Was wurde erstellt:
- **Hauptdatensatz**: `bakery_training_dataset.csv` 
- **Feature-Beschreibung**: `feature_description.csv`
- **Excel-Version**: `bakery_training_dataset.xlsx`

## 🏷️ Enthaltene Features:
1. **Zeitfeatures**: Datum, Jahr, Monat, Tag, Wochentag
2. **Warengruppen**: 6 Kategorien als One-Hot-Encoding
3. **Wetterdaten**: Temperatur, Wind, Bewölkung, Wettercode-Status
4. **Kalender-Events**: Feiertage, Sonntage, Jahreszeiten
5. **Besondere Events**: Kieler Woche
6. **Wirtschaftsdaten**: Preisindex für Backwaren
7. **Zielvariable**: Umsatz in Euro

## 🚀 Nächste Schritte:
1. **Explorative Datenanalyse** (EDA) durchführen
2. **Feature Engineering** verfeinern
3. **Regressionsmodelle** trainieren
4. **Neuronale Netze** implementieren
5. **Modell-Evaluation** und Optimierung

## 💡 Hinweise für das Modelltraining:
- Normalisierung der numerischen Features empfohlen
- Zeitreihen-Splits für Validation verwenden
- Cross-Validation mit Zeitberücksichtigung
- Outlier-Behandlung bei Bedarf anwenden